# Importación de librerias

In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import Normalizer

# Importación de datos

join es la tabla de los prestamos junto a la información de los ítems después de la limpieza y extracción de atributos adicionales. 

In [2]:
#importamos la tabla de join 
join = pd.read_json('https://www.dropbox.com/s/i1komhf7u1c4y95/joinTablas.json?dl=1')

In [3]:
#Eliminamos algunas columnas que no nos interesan para este notebook
join = join.drop(["Fecha","Dewey","Facultad","Temas","Union","TipoItem"], axis=1)
#Ejemplos y columnas del dataframe
display(join.head(3))
display(join.columns)

,RowID,IDItem,NumeroUbicacion,Ubicacion,Llave,Programa,IDUsuario,Year,Signatura,Autor,Titulo,AnioPublicacion,DeweyEspecifico,TemaDewey,DeweyUnidad,DeweyDecena,DeweyCentena
0,Row0,80000005327627,720.98 A71S,COL-GRAL-2,866245,CARRERA DE ARQUITECTURA,c361c772cd0220bb16dcfb2ad803e4903334ab94,2019,720.98,"Arango Cardinal, Silvia 1948-",Ciudad y arquitectura seis generaciones que co...,2012.0,720.98,Arquitectura latinoamericana,720,720,700
1,Row1,80000001191496,720.9861 A71,COL-GRAL-2,309945,CARRERA DE ARQUITECTURA,c361c772cd0220bb16dcfb2ad803e4903334ab94,2019,720.9861,"Arango Cardinal, Silvia 1948-",Historia de la arquitectura en Colombia Silvia...,1993.0,720.9861,Arquitectura colombiana,720,720,700
2,Row2,80000004979759,540 CH15Q 2010,COL-GRAL-3,822727,CARRERA DE ARQUITECTURA,87b0e5a61ed712ddfaf5d478ad68c87c825997e9,2019,540,"Chang, Raymond",Química Raymond Chang ; revisión técnica Rosa ...,2010.0,540,Química,540,540,500


Index(['RowID', 'IDItem', 'NumeroUbicacion', 'Ubicacion', 'Llave', 'Programa',
       'IDUsuario', 'Year', 'Signatura', 'Autor', 'Titulo', 'AnioPublicacion',
       'DeweyEspecifico', 'TemaDewey', 'DeweyUnidad', 'DeweyDecena',
       'DeweyCentena'],
      dtype='object')

In [4]:
join.loc[join.IDUsuario == "6466dbb15c41fdacb59eb1179817958de2c57191"].shape[0]


17

# Preparación de los datos
Crearemos la columna de pesos así como el dataframe que identificará los gustos y preferencias de los usuarios. Estos dataframes se crearán teniendo en cuenta 3 columnas diferentes:
* DeweyUnidad 
* DeweyDecena
* DeweyCentena


**Creación de columna "Peso"**:
A partir del análisis con expertos en negocio, se ha entendido que es importante tomar en cuenta el cambio de gustos de los usuarios a través del tiempo y dar menos peso a prestamos que se realizaron en el pasado a prestamos más recientes. Para esto se creará una nueva columna denominada "Peso" la cual determinará el peso quue tiene dicho prestamo.
La disminución del peso será exponencial según la diferencia con el año actual y se calculará con la siguiente formula:

$Peso = \frac{1}{2^{(2021-20xx)}}$


In [5]:
#Creamos la columna pesos para el dataframe a partir del año en que se realizó el prestamo
join["Peso"] = join.apply(lambda row: 1/2**(2021-row.Year), axis=1 )

In [6]:
join.loc[join.IDUsuario == "6466dbb15c41fdacb59eb1179817958de2c57191"]


,RowID,IDItem,NumeroUbicacion,Ubicacion,Llave,Programa,IDUsuario,Year,Signatura,Autor,Titulo,AnioPublicacion,DeweyEspecifico,TemaDewey,DeweyUnidad,DeweyDecena,DeweyCentena,Peso
159618,Row174866,80000004724397,813 P15CA,COL-GRAL-0,797606,CARRERA DE INGENIERÍA DE SISTEMAS,6466dbb15c41fdacb59eb1179817958de2c57191,2019,813,"Palahniuk, Chuck 1962-",Asfixia Chuck Palahniuk ; traducción de Javier...,2009.0,813,Novela histórica estadounidense,813,810,800,0.25
159621,Row174869,80000002595877,813 D35DR,COL-GRAL-0,349693,CARRERA DE INGENIERÍA DE SISTEMAS,6466dbb15c41fdacb59eb1179817958de2c57191,2019,813,"DeLillo, Don 1936-",Ruido de fondo Don DeLillo ; traducción de Gia...,1994.0,813,Novela histórica estadounidense,813,810,800,0.25
160653,Row175959,80000002595877,813 D35DR,COL-GRAL-0,349693,CARRERA DE INGENIERÍA DE SISTEMAS,6466dbb15c41fdacb59eb1179817958de2c57191,2019,813,"DeLillo, Don 1936-",Ruido de fondo Don DeLillo ; traducción de Gia...,1994.0,813,Novela histórica estadounidense,813,810,800,0.25
161597,Row176919,80000003726378,823 G73M,COL-GRAL-0,277628,CARRERA DE INGENIERÍA DE SISTEMAS,6466dbb15c41fdacb59eb1179817958de2c57191,2019,823,"Greene, Graham 1904-1991",The ministry of fear Graham Greene,1963.0,823,Novela tanzana (Inglés),823,820,800,0.25
162412,Row177762,80000002595877,813 D35DR,COL-GRAL-0,349693,CARRERA DE INGENIERÍA DE SISTEMAS,6466dbb15c41fdacb59eb1179817958de2c57191,2019,813,"DeLillo, Don 1936-",Ruido de fondo Don DeLillo ; traducción de Gia...,1994.0,813,Novela histórica estadounidense,813,810,800,0.25
164093,Row179569,80000003726378,823 G73M,COL-GRAL-0,277628,CARRERA DE INGENIERÍA DE SISTEMAS,6466dbb15c41fdacb59eb1179817958de2c57191,2019,823,"Greene, Graham 1904-1991",The ministry of fear Graham Greene,1963.0,823,Novela tanzana (Inglés),823,820,800,0.25
165231,Row180745,80000003726378,823 G73M|,COL-GRAL-0,277628,CARRERA DE INGENIERÍA DE SISTEMAS,6466dbb15c41fdacb59eb1179817958de2c57191,2019,823,"Greene, Graham 1904-1991",The ministry of fear Graham Greene,1963.0,823,Novela tanzana (Inglés),823,820,800,0.25
166171,Row181719,80000004687401,621.3893 R47|,COL-GRAL-2,766719,CARRERA DE INGENIERÍA DE SISTEMAS,6466dbb15c41fdacb59eb1179817958de2c57191,2019,621.3893,"Miranda, Eduardo Reck 1963-",Computer sound design synthesis techniques and...,2002.0,621.3893,Sistemas de sonido envolvente,621,620,600,0.25
166172,Row181720,80000005077492,813 A74IN|,COL-GRAL-0,838306,CARRERA DE INGENIERÍA DE SISTEMAS,6466dbb15c41fdacb59eb1179817958de2c57191,2019,813,"Asimov, Isaac 1920-1992.",Nueve futuros Isaac Asimov ; traducción de Cés...,1990.0,813,Novela histórica estadounidense,813,810,800,0.25
166173,Row181721,80000005133846,813 B87WNE|,COL-GRAL-0,844792,CARRERA DE INGENIERÍA DE SISTEMAS,6466dbb15c41fdacb59eb1179817958de2c57191,2019,813,"Burroughs, William Seward 1914-1997",El almuerzo desnudo William S. Burroughs ; tra...,2011.0,813,Novela histórica estadounidense,813,810,800,0.25


La siguiente función crea la tabla de preferencias y gustos de los usuarios a partir de los prestamos y la fecha de estos(Columna Peso).

Cada Fila representa un usuario, las columnas son los deweys y los datos es el nivel de pertenencia del usuario frente a dicho dewey.

In [7]:
agrupacion = join.groupby(["IDUsuario","DeweyUnidad"])["Peso"].sum().reset_index(name="Peso")
agrupacion

,IDUsuario,DeweyUnidad,Peso
0,00063d52cf68c65d2a569e95c40345c4a305ccc7,658,0.500
1,0006b3bdabeaa5389b211c8487bd67d339c97ef7,378,0.125
2,0006b3bdabeaa5389b211c8487bd67d339c97ef7,657,0.375
3,000a0630c32a437052696f6921e3181f155291d0,1,0.500
4,000c040064aedd76057c5309064a3823db970644,657,0.125
...,...,...,...
125648,fff735f54719b720fc4c6c723208ad1f06e9605e,986,0.375
125649,fffb2be68fef4b27a810b11a641d84052eb02347,701,0.125
125650,fffb2be68fef4b27a810b11a641d84052eb02347,776,0.125
125651,fffb2be68fef4b27a810b11a641d84052eb02347,781,0.250


In [8]:
#crearTablaPesos: crea la tabla de pesos por usuario
#parámetros
    #columna = {deweyUnidad|deweyDecena|deweyCentena}
    #nombreArchivo = nombre con el que se va a exportar el archivo
def crearTablaPesos(columna):
    #Tabla = pesos_usuarios
        #columnas = deweys
        #filas = usuarios
        #dato = peso que tiene el usuario en dicho dewey
    #creamos las columnas a partir de los deweys diferentes. 
    agrupacion = join.groupby(["IDUsuario",columna])["Peso"].sum().reset_index(name="Peso")
    display(agrupacion)
    
    pesos_usuarios = pd.DataFrame()
    aux = pd.DataFrame()
    ids = agrupacion["IDUsuario"].unique()
    print(len(ids))
    for usuario in ids:
        #obtenemos todos los prestamos del usuario
        prestamos = agrupacion.loc[agrupacion["IDUsuario"]==usuario]
        columnas = prestamos[columna].values
        pesos = prestamos["Peso"].values
        fila = pd.DataFrame(data = [pesos], columns = columnas)
        fila["IDUsuario"] = usuario
        #pesos_usuarios = pesos_usuarios.append(fila)
        aux = pd.concat([aux,fila])
        if(aux.shape[0] == 100):
            pesos_usuarios = pd.concat([pesos_usuarios,aux])
            aux = pd.DataFrame()
    pesos_usuarios = pd.concat([pesos_usuarios,aux])
    pesos_usuarios = pesos_usuarios.fillna(0)
    pesos_usuarios.reset_index(drop=True, inplace=True)
    return pesos_usuarios

Se crean 3 tablas de peso diferentes según la columna que se tuvo en cuenta:

{deweyUnidad|deweyDecena|deweyCentena}

In [9]:
pesos_usuarios_unidad=crearTablaPesos("DeweyUnidad")

,IDUsuario,DeweyUnidad,Peso
0,00063d52cf68c65d2a569e95c40345c4a305ccc7,658,0.500
1,0006b3bdabeaa5389b211c8487bd67d339c97ef7,378,0.125
2,0006b3bdabeaa5389b211c8487bd67d339c97ef7,657,0.375
3,000a0630c32a437052696f6921e3181f155291d0,1,0.500
4,000c040064aedd76057c5309064a3823db970644,657,0.125
...,...,...,...
125648,fff735f54719b720fc4c6c723208ad1f06e9605e,986,0.375
125649,fffb2be68fef4b27a810b11a641d84052eb02347,701,0.125
125650,fffb2be68fef4b27a810b11a641d84052eb02347,776,0.125
125651,fffb2be68fef4b27a810b11a641d84052eb02347,781,0.250


24407


In [10]:
pesos_usuarios_decena = crearTablaPesos("DeweyDecena")

,IDUsuario,DeweyDecena,Peso
0,00063d52cf68c65d2a569e95c40345c4a305ccc7,650,0.500
1,0006b3bdabeaa5389b211c8487bd67d339c97ef7,370,0.125
2,0006b3bdabeaa5389b211c8487bd67d339c97ef7,650,0.375
3,000a0630c32a437052696f6921e3181f155291d0,0,0.500
4,000c040064aedd76057c5309064a3823db970644,650,0.125
...,...,...,...
96592,fff735f54719b720fc4c6c723208ad1f06e9605e,980,0.375
96593,fffb2be68fef4b27a810b11a641d84052eb02347,700,0.125
96594,fffb2be68fef4b27a810b11a641d84052eb02347,770,0.125
96595,fffb2be68fef4b27a810b11a641d84052eb02347,780,0.250


24407


In [11]:
pesos_usuarios_centena = crearTablaPesos("DeweyCentena")

,IDUsuario,DeweyCentena,Peso
0,00063d52cf68c65d2a569e95c40345c4a305ccc7,600,0.500
1,0006b3bdabeaa5389b211c8487bd67d339c97ef7,300,0.125
2,0006b3bdabeaa5389b211c8487bd67d339c97ef7,600,0.375
3,000a0630c32a437052696f6921e3181f155291d0,0,0.500
4,000c040064aedd76057c5309064a3823db970644,600,0.125
...,...,...,...
61835,fff735f54719b720fc4c6c723208ad1f06e9605e,600,2.625
61836,fff735f54719b720fc4c6c723208ad1f06e9605e,800,2.875
61837,fff735f54719b720fc4c6c723208ad1f06e9605e,900,0.375
61838,fffb2be68fef4b27a810b11a641d84052eb02347,700,0.500


24407


In [12]:
pesos_usuarios_centena

,600,IDUsuario,300,0,200,800,700,100,400,500,900,-900
0,0.500,00063d52cf68c65d2a569e95c40345c4a305ccc7,0.000,0.000,0.0,0.000,0.0,0.000,0.0,0.0,0.000,0.0
1,0.375,0006b3bdabeaa5389b211c8487bd67d339c97ef7,0.125,0.000,0.0,0.000,0.0,0.000,0.0,0.0,0.000,0.0
2,0.000,000a0630c32a437052696f6921e3181f155291d0,0.000,0.500,0.0,0.000,0.0,0.000,0.0,0.0,0.000,0.0
3,0.125,000c040064aedd76057c5309064a3823db970644,0.000,0.000,0.0,0.000,0.0,0.000,0.0,0.0,0.000,0.0
4,0.000,000f23e62e6d4995ede7090c6447ad6736edc4de,1.000,0.000,0.5,0.000,0.0,0.000,0.0,0.0,0.000,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...
24402,0.000,fff71390a9f54ced5134452870d70ff18812f05f,2.125,0.000,0.0,0.000,0.0,0.000,0.0,0.0,0.000,0.0
24403,0.000,fff716c9685b5267e6ecfcdc00e9303509355bc1,1.000,0.000,0.0,0.000,0.0,0.000,0.0,0.0,0.000,0.0
24404,2.625,fff735f54719b720fc4c6c723208ad1f06e9605e,7.375,0.125,0.0,2.875,0.0,0.125,0.0,0.5,0.375,0.5
24405,0.000,fffb2be68fef4b27a810b11a641d84052eb02347,0.000,0.000,0.0,0.000,0.5,0.000,0.0,0.0,0.000,0.0


Exportamos e Importamos las tablas para agilizar el proceso

In [13]:
pesos_usuarios_unidad.to_json(r'C:\Users\user\Downloads\pesos_usuario_x_dewey_unidad.json')
pesos_usuarios_decena.to_json(r'C:\Users\user\Downloads\pesos_usuario_x_dewey_decena.json')
pesos_usuarios_centena.to_json(r'C:\Users\user\Downloads\pesos_usuario_x_dewey_centena.json')

In [14]:
pesos_usuarios_unidad = pd.read_json('https://www.dropbox.com/s/aitygqwn9q47rlg/pesos_usuario_x_dewey_unidad.json?dl=1')

In [15]:
pesos_usuarios_decena = pd.read_json('https://www.dropbox.com/s/vr6ehn8xjhojuba/pesos_usuario_x_dewey_decena.json?dl=1')

In [16]:
pesos_usuarios_centena = pd.read_json('https://www.dropbox.com/s/2vnntgjnqpijkgg/pesos_usuario_x_dewey_centena.json?dl=1')

# Normalización de pesos
En busca de que dos usuarios con mismos gustos pero diferente cantidad de prestamos sean equivalentes se decidió  por normalizar cada una de las filas. Así un usuario con 3 prestamos en el dewey 600 y otro con un único prestamo en el dewey 600 se velven equivalente al tener gustos únicamente por este dewey.

In [18]:
#función: normalizar las filas por valores entre cero y uno.
    #Parámetros: pesos_usuario, representa la matriz dispersa que se va a normalizar.
    #{pesos_usuarios_unidad|pesos_usuarios_decena|pesos_usuarios_centena}
def normalizar_pesos(pesos_usuario):
    print("Normalizando pesos...Iniciando")
    usuarios = pesos_usuario['IDUsuario']
    pesos_usuario = pesos_usuario.apply(pd.to_numeric, errors='coerce').drop(["IDUsuario"],axis=1)
    display(pesos_usuario)
    #Normalizamos por fila
    sumatoria = pesos_usuario.max(axis=1)
    pesos_norm = pesos_usuario.div(pesos_usuario.sum(axis=1), axis=0)
    pesos_norm_id = pesos_norm.copy()
    # a la tabla le agregamos la columna de IDUsuario para poder identificar que pesos son de cada usuario
    if len(pesos_norm_id) == len(join.IDUsuario.unique()):
        pesos_norm_id['IDUsuario'] = usuarios
        display(pesos_norm_id.loc[pesos_norm_id["IDUsuario"] == "6466dbb15c41fdacb59eb1179817958de2c57191"])
    print("Normalizando pesos...Acabado")
    return pesos_norm_id

In [19]:
pesos_norm_id_unidad = normalizar_pesos(pesos_usuarios_unidad)
display(pesos_norm_id_unidad.shape)
pesos_norm_id_unidad.head(10)

Normalizando pesos...Iniciando


,658,378,657,1,227,347,350,808,812,813,...,964,367,383,494,413,496,799,855,876,97
0,0.5,0.000,0.000,0.000,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0
1,0.0,0.125,0.375,0.000,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0
2,0.0,0.000,0.000,0.500,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0
3,0.0,0.000,0.125,0.000,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0
4,0.0,0.000,0.000,0.000,0.5,0.5,0.5,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24402,0.0,0.000,0.000,0.000,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0
24403,0.0,0.000,0.000,0.000,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0
24404,0.5,0.000,0.000,0.125,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0
24405,0.0,0.000,0.000,0.000,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0


,658,378,657,1,227,347,350,808,812,813,...,367,383,494,413,496,799,855,876,97,IDUsuario
9664,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.705882,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,6466dbb15c41fdacb59eb1179817958de2c57191


Normalizando pesos...Acabado


(24407, 756)

,658,378,657,1,227,347,350,808,812,813,...,367,383,494,413,496,799,855,876,97,IDUsuario
0,1.000000,0.00,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,00063d52cf68c65d2a569e95c40345c4a305ccc7
1,0.000000,0.25,0.750000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0006b3bdabeaa5389b211c8487bd67d339c97ef7
2,0.000000,0.00,0.000000,1.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,000a0630c32a437052696f6921e3181f155291d0
3,0.000000,0.00,1.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,000c040064aedd76057c5309064a3823db970644
4,0.000000,0.00,0.000000,0.0,0.333333,0.333333,0.333333,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,000f23e62e6d4995ede7090c6447ad6736edc4de
5,0.000000,0.00,0.000000,0.0,0.000000,0.000000,0.000000,0.166667,0.166667,0.666667,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,00110738ad91d1cfb52f4c1fe3fa0e7c9b3e277e
6,0.461538,0.00,0.153846,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,00159e086af7d72d80602291964442a68472ed39
7,0.000000,0.00,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,00186f0d22b83441cca048ddf1e6eab99cc95b7b
8,0.000000,0.00,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,001c22ce26824deb5633eab2d4572e1a527b76e2
9,0.000000,0.00,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,001c411a83261e3123c261cc392bd59ce2237b18


In [20]:
pesos_norm_id_decena = normalizar_pesos(pesos_usuarios_decena)
display(pesos_norm_id_decena.shape)
pesos_norm_id_decena.head(3)

Normalizando pesos...Iniciando


,650,370,0,220,340,350,800,810,20,330,...,560,640,470,310,10,480,90,990,50,30
0,0.500,0.000,0.000,0.0,0.000,0.00,0.0,0.0,0.0,0.000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.375,0.125,0.000,0.0,0.000,0.00,0.0,0.0,0.0,0.000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.000,0.000,0.500,0.0,0.000,0.00,0.0,0.0,0.0,0.000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.125,0.000,0.000,0.0,0.000,0.00,0.0,0.0,0.0,0.000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.000,0.000,0.000,0.5,0.500,0.50,0.0,0.0,0.0,0.000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24402,0.000,0.000,0.000,0.0,0.000,0.00,0.0,0.0,0.0,0.875,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
24403,0.000,0.000,0.000,0.0,1.000,0.00,0.0,0.0,0.0,0.000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
24404,0.500,0.500,0.125,0.0,0.375,0.25,0.0,0.0,0.0,0.375,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
24405,0.000,0.000,0.000,0.0,0.000,0.00,0.0,0.0,0.0,0.000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


,650,370,0,220,340,350,800,810,20,330,...,640,470,310,10,480,90,990,50,30,IDUsuario
9664,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.705882,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,6466dbb15c41fdacb59eb1179817958de2c57191


Normalizando pesos...Acabado


(24407, 100)

,650,370,0,220,340,350,800,810,20,330,...,640,470,310,10,480,90,990,50,30,IDUsuario
0,1.00,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,00063d52cf68c65d2a569e95c40345c4a305ccc7
1,0.75,0.25,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0006b3bdabeaa5389b211c8487bd67d339c97ef7
2,0.00,0.00,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,000a0630c32a437052696f6921e3181f155291d0


In [21]:
pesos_norm_id_centena = normalizar_pesos(pesos_usuarios_centena)
display(pesos_norm_id_centena.shape)
pesos_norm_id_centena.head(3)

Normalizando pesos...Iniciando


,600,300,0,200,800,700,100,400,500,900,-900
0,0.500,0.000,0.000,0.0,0.000,0.0,0.000,0.0,0.0,0.000,0.0
1,0.375,0.125,0.000,0.0,0.000,0.0,0.000,0.0,0.0,0.000,0.0
2,0.000,0.000,0.500,0.0,0.000,0.0,0.000,0.0,0.0,0.000,0.0
3,0.125,0.000,0.000,0.0,0.000,0.0,0.000,0.0,0.0,0.000,0.0
4,0.000,1.000,0.000,0.5,0.000,0.0,0.000,0.0,0.0,0.000,0.0
...,...,...,...,...,...,...,...,...,...,...,...
24402,0.000,2.125,0.000,0.0,0.000,0.0,0.000,0.0,0.0,0.000,0.0
24403,0.000,1.000,0.000,0.0,0.000,0.0,0.000,0.0,0.0,0.000,0.0
24404,2.625,7.375,0.125,0.0,2.875,0.0,0.125,0.0,0.5,0.375,0.5
24405,0.000,0.000,0.000,0.0,0.000,0.5,0.000,0.0,0.0,0.000,0.0


,600,300,0,200,800,700,100,400,500,900,-900,IDUsuario
9664,0.058824,0.0,0.0,0.0,0.941176,0.0,0.0,0.0,0.0,0.0,0.0,6466dbb15c41fdacb59eb1179817958de2c57191


Normalizando pesos...Acabado


(24407, 12)

,600,300,0,200,800,700,100,400,500,900,-900,IDUsuario
0,1.00,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,00063d52cf68c65d2a569e95c40345c4a305ccc7
1,0.75,0.25,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0006b3bdabeaa5389b211c8487bd67d339c97ef7
2,0.00,0.00,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,000a0630c32a437052696f6921e3181f155291d0


# Agrupación disfusa
Para la agrupación cada usuario podrá pertenecer a uno o mas deweys. Cada usuario tendra un grado de pertenencia a un dewey en especifico dependiendo de sus gustos y preferencias resumidos en las matrices dispersas. 
La función se debe recorrer mínimo 3 veces con diferentes matrices de dispersión:
* pesos_usuarios_unidad
* pesos_usuarios_decena
* pesos_usuarios_centena

In [22]:
#Elimina los valores diferentes de cero para una fila especifica
#Sirve para visualizar únicamente los deweys donde el usuario tiene prestamos. 
def eliminar_cero(id_usuario, df_pesos):
    m1 = (df_pesos['IDUsuario'] == id_usuario)
    m2 = (df_pesos[m1] != 0).all()
    return df_pesos.loc[m1,m2]

In [23]:
eliminar_cero("33c7a2220802831409c62333e13f068363a5d768", pesos_norm_id_unidad)

,1,517,5,IDUsuario
4945,0.333333,0.333333,0.333333,33c7a2220802831409c62333e13f068363a5d768


In [24]:
#Esta función asocia una grado de pertenencia
#Parámetros:
    #pesos_usuarios: matrix dispersa de pesos {pesos_usuarios_unidad|pesos_usuarios_decena|pesos_usuarios_centena}
    #nombre_archivo: nombre del archivo a exportar con la tabla de df_pertenencia
#se crea la tabla df_pertenencia: describe la pertenecia de cada usuario a un cluster(dewey)
def clustering(pesos_usuarios):
    print("Comenzando clustering...")
    #normalizamos los pesos
    pesos_norm_id_unidad = normalizar_pesos(pesos_usuarios)
    #ponemos el usuario a ambos dataframes
    pesos_usuarios["IDUsuario"] = pesos_norm_id_unidad["IDUsuario"]
    #Creamos la estructura del dataframe
    df_pertenencia = pd.DataFrame(columns = ['IDUsuario', 'Cluster', 'Pertenencia','Peso'])
    #iteramos por cada fila(usuario) del dataframe
    for index, row in pesos_norm_id_unidad.iterrows():
        #Extraemos los usuarios
        usuario = row["IDUsuario"]
        #Extraemos los pesos deiferentes de cero normalizados y sin normalizar
        usuario_limpio_norm = eliminar_cero(row["IDUsuario"], pesos_norm_id_unidad)
        usuario_limpio_orig = eliminar_cero(row["IDUsuario"], pesos_usuarios)
        pertenecias = usuario_limpio_norm.drop(["IDUsuario"], axis=1).values
        pesos = usuario_limpio_orig.drop(["IDUsuario"], axis=1).values
        #los clusters son las columnas de estos dataframes
        clusters = usuario_limpio_norm.drop(["IDUsuario"], axis=1).columns
        #creamos un array que tenga las veces necesarias al usuario
        tamanio = len(clusters)
        usuarios = np.repeat([usuario], tamanio)
        #agregamos la fila al dataframe
        fila = pd.DataFrame(data = {'IDUsuario': usuarios, 'Cluster': clusters
                                   , 'Pertenencia': pertenecias[0], 'Peso': pesos[0]})
        df_pertenencia = df_pertenencia.append(fila)
    print("Finalizando clustering...")
    return df_pertenencia

In [25]:
pesos_clustering_unidad = clustering(pesos_usuarios_unidad)
display(pesos_clustering_unidad.head(3))

Comenzando clustering...
Normalizando pesos...Iniciando


,658,378,657,1,227,347,350,808,812,813,...,964,367,383,494,413,496,799,855,876,97
0,0.5,0.000,0.000,0.000,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0
1,0.0,0.125,0.375,0.000,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0
2,0.0,0.000,0.000,0.500,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0
3,0.0,0.000,0.125,0.000,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0
4,0.0,0.000,0.000,0.000,0.5,0.5,0.5,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24402,0.0,0.000,0.000,0.000,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0
24403,0.0,0.000,0.000,0.000,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0
24404,0.5,0.000,0.000,0.125,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0
24405,0.0,0.000,0.000,0.000,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0


,658,378,657,1,227,347,350,808,812,813,...,367,383,494,413,496,799,855,876,97,IDUsuario
9664,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.705882,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,6466dbb15c41fdacb59eb1179817958de2c57191


Normalizando pesos...Acabado
Finalizando clustering...


,IDUsuario,Cluster,Pertenencia,Peso
0,00063d52cf68c65d2a569e95c40345c4a305ccc7,658,1.00,0.500
0,0006b3bdabeaa5389b211c8487bd67d339c97ef7,378,0.25,0.125
1,0006b3bdabeaa5389b211c8487bd67d339c97ef7,657,0.75,0.375


In [26]:
pesos_clustering_decena = clustering(pesos_usuarios_decena)
display(pesos_clustering_decena.head(3))

Comenzando clustering...
Normalizando pesos...Iniciando


,650,370,0,220,340,350,800,810,20,330,...,560,640,470,310,10,480,90,990,50,30
0,0.500,0.000,0.000,0.0,0.000,0.00,0.0,0.0,0.0,0.000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.375,0.125,0.000,0.0,0.000,0.00,0.0,0.0,0.0,0.000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.000,0.000,0.500,0.0,0.000,0.00,0.0,0.0,0.0,0.000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.125,0.000,0.000,0.0,0.000,0.00,0.0,0.0,0.0,0.000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.000,0.000,0.000,0.5,0.500,0.50,0.0,0.0,0.0,0.000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24402,0.000,0.000,0.000,0.0,0.000,0.00,0.0,0.0,0.0,0.875,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
24403,0.000,0.000,0.000,0.0,1.000,0.00,0.0,0.0,0.0,0.000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
24404,0.500,0.500,0.125,0.0,0.375,0.25,0.0,0.0,0.0,0.375,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
24405,0.000,0.000,0.000,0.0,0.000,0.00,0.0,0.0,0.0,0.000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


,650,370,0,220,340,350,800,810,20,330,...,640,470,310,10,480,90,990,50,30,IDUsuario
9664,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.705882,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,6466dbb15c41fdacb59eb1179817958de2c57191


Normalizando pesos...Acabado
Finalizando clustering...


,IDUsuario,Cluster,Pertenencia,Peso
0,00063d52cf68c65d2a569e95c40345c4a305ccc7,650,1.00,0.500
0,0006b3bdabeaa5389b211c8487bd67d339c97ef7,650,0.75,0.375
1,0006b3bdabeaa5389b211c8487bd67d339c97ef7,370,0.25,0.125


In [27]:
pesos_clustering_centena = clustering(pesos_usuarios_centena)
display(pesos_clustering_centena.head(3))

Comenzando clustering...
Normalizando pesos...Iniciando


,600,300,0,200,800,700,100,400,500,900,-900
0,0.500,0.000,0.000,0.0,0.000,0.0,0.000,0.0,0.0,0.000,0.0
1,0.375,0.125,0.000,0.0,0.000,0.0,0.000,0.0,0.0,0.000,0.0
2,0.000,0.000,0.500,0.0,0.000,0.0,0.000,0.0,0.0,0.000,0.0
3,0.125,0.000,0.000,0.0,0.000,0.0,0.000,0.0,0.0,0.000,0.0
4,0.000,1.000,0.000,0.5,0.000,0.0,0.000,0.0,0.0,0.000,0.0
...,...,...,...,...,...,...,...,...,...,...,...
24402,0.000,2.125,0.000,0.0,0.000,0.0,0.000,0.0,0.0,0.000,0.0
24403,0.000,1.000,0.000,0.0,0.000,0.0,0.000,0.0,0.0,0.000,0.0
24404,2.625,7.375,0.125,0.0,2.875,0.0,0.125,0.0,0.5,0.375,0.5
24405,0.000,0.000,0.000,0.0,0.000,0.5,0.000,0.0,0.0,0.000,0.0


,600,300,0,200,800,700,100,400,500,900,-900,IDUsuario
9664,0.058824,0.0,0.0,0.0,0.941176,0.0,0.0,0.0,0.0,0.0,0.0,6466dbb15c41fdacb59eb1179817958de2c57191


Normalizando pesos...Acabado
Finalizando clustering...


,IDUsuario,Cluster,Pertenencia,Peso
0,00063d52cf68c65d2a569e95c40345c4a305ccc7,600,1.00,0.500
0,0006b3bdabeaa5389b211c8487bd67d339c97ef7,600,0.75,0.375
1,0006b3bdabeaa5389b211c8487bd67d339c97ef7,300,0.25,0.125


In [28]:
pesos_clustering_unidad.reset_index(drop=True, inplace=True)
pesos_clustering_decena.reset_index(drop=True, inplace=True)
pesos_clustering_centena.reset_index(drop=True, inplace=True)

In [29]:
pesos_clustering_unidad.to_json(r'C:\Users\user\Downloads\pesos_clustering_unidad.json')
pesos_clustering_decena.to_json(r'C:\Users\user\Downloads\pesos_clustering_decena.json')
pesos_clustering_centena.to_json(r'C:\Users\user\Downloads\pesos_clustering_centena.json')

In [30]:
pesos_clustering_unidad = pd.read_json('https://www.dropbox.com/s/6j30n8y3fn8358l/pesos_clustering_unidad.json?dl=1')
pesos_clustering_decena = pd.read_json('https://www.dropbox.com/s/6m7vbpfq8b8qz4s/pesos_clustering_decena.json?dl=1')
pesos_clustering_centena = pd.read_json('https://www.dropbox.com/s/3rjqco5swu55cna/pesos_clustering_centena.json?dl=1')

In [31]:
display(pesos_clustering_unidad.head(20))
print(pesos_clustering_unidad.shape)

,IDUsuario,Cluster,Pertenencia,Peso
0,00063d52cf68c65d2a569e95c40345c4a305ccc7,658,1.000000,0.500
1,0006b3bdabeaa5389b211c8487bd67d339c97ef7,378,0.250000,0.125
2,0006b3bdabeaa5389b211c8487bd67d339c97ef7,657,0.750000,0.375
3,000a0630c32a437052696f6921e3181f155291d0,1,1.000000,0.500
4,000c040064aedd76057c5309064a3823db970644,657,1.000000,0.125
5,000f23e62e6d4995ede7090c6447ad6736edc4de,227,0.333333,0.500
6,000f23e62e6d4995ede7090c6447ad6736edc4de,347,0.333333,0.500
7,000f23e62e6d4995ede7090c6447ad6736edc4de,350,0.333333,0.500
8,00110738ad91d1cfb52f4c1fe3fa0e7c9b3e277e,808,0.166667,0.125
9,00110738ad91d1cfb52f4c1fe3fa0e7c9b3e277e,812,0.166667,0.125


(125653, 4)


In [32]:
pesos_clustering_unidad.loc[pesos_clustering_unidad.IDUsuario == "6466dbb15c41fdacb59eb1179817958de2c57191"]


,IDUsuario,Cluster,Pertenencia,Peso
49724,6466dbb15c41fdacb59eb1179817958de2c57191,813,0.705882,3.00
49725,6466dbb15c41fdacb59eb1179817958de2c57191,823,0.176471,0.75
49726,6466dbb15c41fdacb59eb1179817958de2c57191,621,0.058824,0.25
49727,6466dbb15c41fdacb59eb1179817958de2c57191,833,0.058824,0.25


# Recomendaciones


La función rank_material rankea el material de un dewey en especifico a partir de la experticie de los usuarios dentro de dicho dewey. A partir de los pesos sabemos que tanto sabe un usuario acerca de un dewey especifico. Normalizamos estos valores y a cada libro alquilado que tenga el usuario x al libro x se le da ese puntaje. El puntaje total de un libro es igual a la sumatoria de los puntajes dados por los usuarios. 

In [44]:
#Parámetros:
 #cluster_pertenencia:dataframe con la pertenencia de los usuarios a un cluster.
 #cluster: cluster a evaluar
 #columna: define la columna sobre la cual se va a realizar el join {DeweyUnidad|DeweyDecena|DeweyCentena}
def rank_material(pesos_clustering, cluster, columna):
    #Buscamos únicamente a los usuarios que tienen un grado de pertenencia al cluster.
    usuarios = pesos_clustering.loc[pesos_clustering.Cluster == cluster]
    usuarios["Peso"] = usuarios["Peso"].apply(lambda x: float(x))
    #si el peso de algún usuario es mayor a 10 este se deja con el valor de 10
    usuarios.loc[usuarios.Peso > 10, "Peso"] = 10
    
    #normalizar columna de peso
    max_value = usuarios["Peso"].max()
    min_value = usuarios["Peso"].min()
    usuarios["Peso"] = (usuarios["Peso"]) / (max_value)
    #filtramos de la tabla de join los rows con el dewey
    prestamos = join.loc[join[columna] == int(cluster)]
    #display(prestamos)
    #join
    prestamos_data = pd.DataFrame(data=prestamos)
    usuario_data = pd.DataFrame(data=usuarios)
    join_tablas = pd.merge(prestamos_data, usuario_data, left_on='IDUsuario', right_on='IDUsuario', how='inner')
    #finalmente agrupamos al material por llave y hacemos la sumatoria de sus pesos
    materialRank = join_tablas.groupby(["Llave"])["Peso_y"].sum().reset_index(name="Peso")#Peso o Peso_y
    materialRank = materialRank.sort_values(by=["Peso"],ascending=False)
    return materialRank


In [45]:
material_rankeado = rank_material(pesos_clustering_unidad, 574, "DeweyUnidad")
material_rankeado

<ipython-input-44-d043a82d3f93>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  usuarios["Peso"] = usuarios["Peso"].apply(lambda x: float(x))
C:\Users\user\anaconda3\lib\site-packages\pandas\core\indexing.py:1720: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)
<ipython-input-44-d043a82d3f93>:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the

,Llave,Peso
329,722413,68.8375
429,825335,41.4375
522,884195,40.9625
386,788344,40.5250
396,804821,31.2375
...,...,...
373,769277,0.0125
41,269584,0.0125
213,525939,0.0125
532,894506,0.0125


La función generar recomendación genera todas las recomendaciones para los usuarios de un único cluster teniendo en cuenta el nivel de pertenencia de los usuarios a cada cluster.
Parámetros:
* pesos_clustering: es la matriz generada por la función de clustering
* cluster: cluster del cual se van a sacar las recomendaciones
* peso_x_nivel: peso del nivel de clustering. Los niveles más especificos tienen mayor peso.
* total_recomendaciones: total de recomendaciones que van a recibir los usuarios
* material_rankeado: dataframe generada por la función rank_material.

In [46]:
import math
def generar_recomendacion(pesos_clustering, cluster, peso_x_nivel, total_recomendaciones, material_rankeado):
    recomendaciones = pd.DataFrame(columns = ['IDUsuario', 'Llave', 'Nivel'])
    
    prestamos_cluster = pesos_clustering.loc[pesos_clustering.Cluster == cluster]
    prestamos_cluster["Pertenencia"] = prestamos_cluster["Pertenencia"].astype(float) 
    for index, row in prestamos_cluster.iterrows():
        num_prestamos = math.ceil(row["Pertenencia"] * peso_x_nivel * total_recomendaciones)
        aux = material_rankeado.head(num_prestamos)
        #print(aux.Llave)
        for llave in aux.Llave.iteritems():
            aux_df = pd.DataFrame({'IDUsuario': row.IDUsuario, 'Llave': llave[1], 'Nivel': peso_x_nivel}, index = ["Index 1"])
            #print("Aux: ", aux_df)
            recomendaciones = pd.concat([recomendaciones, aux_df], ignore_index = True)
            #print("Recomendaciones del cluster: ", cluster, recomendaciones.shape[0])
    return recomendaciones

In [47]:
generar_recomendacion(pesos_clustering_unidad, 574, 0.7, 10, material_rankeado)

<ipython-input-46-af4d9565876f>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  prestamos_cluster["Pertenencia"] = prestamos_cluster["Pertenencia"].astype(float)


,IDUsuario,Llave,Nivel
0,0023eb8457e3722d75e8926cb42f9b7fea38283d,722413,0.7
1,0023eb8457e3722d75e8926cb42f9b7fea38283d,825335,0.7
2,0024fc55a484a66241eba9c79c8532c14a981fd8,722413,0.7
3,0024fc55a484a66241eba9c79c8532c14a981fd8,825335,0.7
4,007dc9488f267e0f74d0fd4f0720280da1ac8e5d,722413,0.7
...,...,...,...
3791,ff8164315c407f57567651c4d7d4b83f28a4c69b,788344,0.7
3792,fff169350c34c6e40763143d1643df3d94698cd4,722413,0.7
3793,fff169350c34c6e40763143d1643df3d94698cd4,825335,0.7
3794,fff169350c34c6e40763143d1643df3d94698cd4,884195,0.7


La función recomendaciones_nivel genera recomendaciones para un nivel completo de clusters. 
Parámetros:
* pesos_clustering: {pesos_clustering_unidad|pesos_clustering_decena|pesos_clustering_centena}
* peso_x_nivel: peso del nivel. Valor entre cero y uno que representa el porcentaje de recomendaciones para dichoi nivel.
* total_recomendaciones: Total de recomendaciones que se le van a hacer a los usuarios.

In [407]:
def recomendaciones_nivel(pesos_clustering, peso_x_nivel, total_recomendaciones):#"DeweyUnidad"
    print("Comenzando Recomendaciones nivel ")
    recomendaciones = pd.DataFrame(columns = ['IDUsuario', 'Llave', 'Nivel'])
    for cluster in pesos_clustering.Cluster.unique():
        cluster = str(cluster)
        material_rankeado = rank_material(pesos_clustering, cluster, "DeweyUnidad")
        #print(material_rankeado)
        lista_recomendaciones = generar_recomendacion(pesos_clustering, cluster, peso_x_nivel, total_recomendaciones, material_rankeado)
        print(lista_recomendaciones.shape[0])
        recomendaciones = pd.concat([recomendaciones, lista_recomendaciones], ignore_index = True)
        #print("Recomendaciones para el grupo: ", cluster, lista_recomendaciones.shape[0])
    print("Finalizando Recomendaciones nivel")
    return recomendaciones

In [408]:
recomendaciones_final_unidad = recomendaciones_nivel(pesos_clustering_unidad, 0.7, 10)
display(recomendaciones_final_decena)

Comenzando Recomendaciones nivel 


<ipython-input-397-d043a82d3f93>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  usuarios["Peso"] = usuarios["Peso"].apply(lambda x: float(x))
C:\Users\user\anaconda3\lib\site-packages\pandas\core\indexing.py:1720: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)
<ipython-input-397-d043a82d3f93>:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in t

13227


<ipython-input-397-d043a82d3f93>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  usuarios["Peso"] = usuarios["Peso"].apply(lambda x: float(x))
C:\Users\user\anaconda3\lib\site-packages\pandas\core\indexing.py:1720: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)
<ipython-input-397-d043a82d3f93>:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in t

706


...


3
1
2
1
1
1
2
1


<ipython-input-401-af4d9565876f>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  prestamos_cluster["Pertenencia"] = prestamos_cluster["Pertenencia"].astype(float)
<ipython-input-397-d043a82d3f93>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  usuarios["Peso"] = usuarios["Peso"].apply(lambda x: float(x))
C:\Users\user\anaconda3\lib\site-packages\pandas\core\indexing.py:1720: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] 

1
1
1
1
1
1
Finalizando Recomendaciones nivel


<ipython-input-397-d043a82d3f93>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  usuarios["Peso"] = usuarios["Peso"].apply(lambda x: float(x))
C:\Users\user\anaconda3\lib\site-packages\pandas\core\indexing.py:1720: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)
<ipython-input-397-d043a82d3f93>:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in t

,IDUsuario,Llave,Nivel


In [409]:
recomendaciones_final_decena = recomendaciones_nivel(pesos_clustering_decena, 0.2, 10)


Comenzando Recomendaciones nivel 


<ipython-input-397-d043a82d3f93>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  usuarios["Peso"] = usuarios["Peso"].apply(lambda x: float(x))
C:\Users\user\anaconda3\lib\site-packages\pandas\core\indexing.py:1720: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)
<ipython-input-397-d043a82d3f93>:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in t

6195


<ipython-input-397-d043a82d3f93>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  usuarios["Peso"] = usuarios["Peso"].apply(lambda x: float(x))
C:\Users\user\anaconda3\lib\site-packages\pandas\core\indexing.py:1720: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)
<ipython-input-397-d043a82d3f93>:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in t

2470
0


...

<ipython-input-397-d043a82d3f93>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  usuarios["Peso"] = usuarios["Peso"].apply(lambda x: float(x))
C:\Users\user\anaconda3\lib\site-packages\pandas\core\indexing.py:1720: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)
<ipython-input-397-d043a82d3f93>:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in t

247
22
227


<ipython-input-397-d043a82d3f93>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  usuarios["Peso"] = usuarios["Peso"].apply(lambda x: float(x))
C:\Users\user\anaconda3\lib\site-packages\pandas\core\indexing.py:1720: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)
<ipython-input-397-d043a82d3f93>:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in t

49
37
43
15
0
0
0
0
Finalizando Recomendaciones nivel


<ipython-input-401-af4d9565876f>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  prestamos_cluster["Pertenencia"] = prestamos_cluster["Pertenencia"].astype(float)
<ipython-input-397-d043a82d3f93>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  usuarios["Peso"] = usuarios["Peso"].apply(lambda x: float(x))
C:\Users\user\anaconda3\lib\site-packages\pandas\core\indexing.py:1720: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] 

In [410]:
recomendaciones_final_centena = recomendaciones_nivel(pesos_clustering_centena, 0.1, 10)
display(recomendaciones_final_centena)

<ipython-input-397-d043a82d3f93>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  usuarios["Peso"] = usuarios["Peso"].apply(lambda x: float(x))
C:\Users\user\anaconda3\lib\site-packages\pandas\core\indexing.py:1720: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)
<ipython-input-397-d043a82d3f93>:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in t

Comenzando Recomendaciones nivel 
10350


<ipython-input-397-d043a82d3f93>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  usuarios["Peso"] = usuarios["Peso"].apply(lambda x: float(x))
C:\Users\user\anaconda3\lib\site-packages\pandas\core\indexing.py:1720: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)
<ipython-input-397-d043a82d3f93>:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in t

12159


<ipython-input-397-d043a82d3f93>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  usuarios["Peso"] = usuarios["Peso"].apply(lambda x: float(x))
C:\Users\user\anaconda3\lib\site-packages\pandas\core\indexing.py:1720: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)
<ipython-input-397-d043a82d3f93>:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in t

...

<ipython-input-397-d043a82d3f93>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  usuarios["Peso"] = usuarios["Peso"].apply(lambda x: float(x))
C:\Users\user\anaconda3\lib\site-packages\pandas\core\indexing.py:1720: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)
<ipython-input-397-d043a82d3f93>:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in t

4435


<ipython-input-397-d043a82d3f93>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  usuarios["Peso"] = usuarios["Peso"].apply(lambda x: float(x))
C:\Users\user\anaconda3\lib\site-packages\pandas\core\indexing.py:1720: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)
<ipython-input-397-d043a82d3f93>:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in t

0
Finalizando Recomendaciones nivel


,IDUsuario,Llave,Nivel
0,00063d52cf68c65d2a569e95c40345c4a305ccc7,313059,0.1
1,0006b3bdabeaa5389b211c8487bd67d339c97ef7,313059,0.1
2,000c040064aedd76057c5309064a3823db970644,313059,0.1
3,00159e086af7d72d80602291964442a68472ed39,313059,0.1
4,00186f0d22b83441cca048ddf1e6eab99cc95b7b,313059,0.1
...,...,...,...
56772,ffa956d8158d97d351f8ee09c9f18aef5289aafb,506507,0.1
56773,ffbb81debf44523b4d21e7f48fcda3e4fbb95f4e,506507,0.1
56774,ffc9639e31d4edee7f1f489137419d65ea15da5e,506507,0.1
56775,ffd24ec2ae25aea321acebccf1e1a08d314461cf,506507,0.1


In [411]:
def recomendaciones_usuario(id_usuario, df_recomendaciones):
    recomendaciones_u = df_recomendaciones.loc[df_recomendaciones["IDUsuario"] == id_usuario]
    display(recomendaciones_u)
    return recomendaciones_u

In [419]:
recomendaciones_tres_niveles = pd.DataFrame(columns = ['IDUsuario', 'Llave', 'Nivel'])
recomendaciones_tres_niveles = pd.concat([recomendaciones_tres_niveles, recomendaciones_final_unidad], ignore_index = True)
recomendaciones_tres_niveles = pd.concat([recomendaciones_tres_niveles, recomendaciones_final_decena], ignore_index = True)
recomendaciones_tres_niveles = pd.concat([recomendaciones_tres_niveles, recomendaciones_final_centena], ignore_index = True)

recomendaciones = recomendaciones_usuario('33c7a2220802831409c62333e13f068363a5d768', recomendaciones_tres_niveles)

,IDUsuario,Llave,Nivel
16984,33c7a2220802831409c62333e13f068363a5d768,918693,0.7
16985,33c7a2220802831409c62333e13f068363a5d768,864918,0.7
16986,33c7a2220802831409c62333e13f068363a5d768,903848,0.7
169677,33c7a2220802831409c62333e13f068363a5d768,258828,0.7
169678,33c7a2220802831409c62333e13f068363a5d768,897767,0.7
169679,33c7a2220802831409c62333e13f068363a5d768,348787,0.7
213066,33c7a2220802831409c62333e13f068363a5d768,903821,0.7
213067,33c7a2220802831409c62333e13f068363a5d768,849385,0.7
213068,33c7a2220802831409c62333e13f068363a5d768,928308,0.7
301533,33c7a2220802831409c62333e13f068363a5d768,846089,0.2


In [420]:
recomendaciones_data = pd.DataFrame(data=recomendaciones)
join_data = pd.DataFrame(data=join)
recomendaciones_completas = pd.merge(recomendaciones_data, join_data, left_on='Llave', right_on='Llave', how='inner')

In [421]:
recomendaciones_completas["Titulo"].unique()

array(['Java how to program early objects Paul Deitel, Harvey Deitel',
       'Ajax, rich internet aplications y desarrollo web para programadores Paul J. Deitel y Harvey M. Deitel ; traductor José Luis Gómez Celador ; editores Víctor Manuel Ruiz Calderón y Susana Krahe Pérez-Rubín',
       'The R book Michael J. Crawley, Imperial college London at Silwood Park, UK.',
       'Programming language pragmatics Michael Lee Scott',
       'Inteligencia artificial fundamentos, práctica y aplicaciones Alberto García Serrano',
       'Hands-on machine learning with Scikit-Learn and TensorFlow concepts, tools, and techniques to build intelligent systems Aurélien Géron',
       'Real analysis Modern techniques and their applications Por Gerald B. Folland',
       'Real analysis Barry Simon',
       'Calculo James Stewart ; traducción Eduardo Ojeda Pena',
       'Arduino curso práctico de formación Óscar Torrente Artero',
       'Process mining data science in action Wil van der Aalst',
       'F

In [1]:
import os
os.system('jupyter nbconvert --to html ClusteringYRecomendacionesDifusas.ipynb')

0

In [13]:
pesos_usuarios_unidad['IDUsuario'] = join.IDUsuario.unique()
pesos_usuarios_decena['IDUsuario'] = join.IDUsuario.unique()
pesos_usuarios_centena['IDUsuario'] = join.IDUsuario.unique()
pesos_usuarios_unidad.columns

Index([        720,         540,         512,         712,         741,
               814,         823,         330,         863,         696,
       ...
               114,          34,         673,         447,         600,
               653,         263,          91,         586, 'IDUsuario'],
      dtype='object', length=756)

In [71]:
def pesos_completos_usuario(id_usuario):
    pesos_unidad = pesos_usuarios_unidad.loc[pesos_usuarios_unidad["IDUsuario"] == id_usuario]
    pesos_decena = pesos_usuarios_decena.loc[pesos_usuarios_decena["IDUsuario"] == id_usuario]
    pesos_centena = pesos_usuarios_centena.loc[pesos_usuarios_centena["IDUsuario"] == id_usuario]
    
    pesos_unidad_copy = pesos_unidad.copy()
    pesos_decena_copy = pesos_decena.copy()
    pesos_centena_copy = pesos_centena.copy()
    
    pesos_unidad_copy = pesos_unidad_copy.drop(labels = ["IDUsuario"], axis = 1)
    pesos_decena_copy = pesos_decena_copy.drop(labels = ["IDUsuario"], axis = 1)
    pesos_centena_copy = pesos_centena_copy.drop(labels = ["IDUsuario"], axis = 1)
    
    #Top Deweys
    
    pesos_unidad_copy = pesos_unidad_copy.T
    pesos_decena_copy = pesos_decena_copy.T
    pesos_centena_copy = pesos_centena_copy.T
    
    pesos_unidad_copy = pesos_unidad_copy.sort_values(by=[0], ascending = False)
    pesos_decena_copy = pesos_decena_copy.sort_values(by=[0], ascending = False)
    pesos_centena_copy = pesos_centena_copy.sort_values(by=[0], ascending = False)
    
    pesos_unidad_copy = pesos_unidad_copy.T
    pesos_decena_copy = pesos_decena_copy.T
    pesos_centena_copy = pesos_centena_copy.T
    
    #Top Dewey normalizado
    
    pesos_unidad_norm = normalizar_pesos(pesos_unidad)
    pesos_decena_norm = normalizar_pesos(pesos_decena)
    pesos_centena_norm = normalizar_pesos(pesos_centena)
    
    pesos_unidad_norm_copy = pesos_unidad_norm.copy()
    pesos_decena_norm_copy = pesos_decena_norm.copy()
    pesos_centena_norm_copy = pesos_centena_norm.copy()
    
    pesos_unidad_norm_copy = pesos_unidad_norm_copy.T
    pesos_decena_norm_copy = pesos_decena_norm_copy.T
    pesos_centena_norm_copy = pesos_centena_norm_copy.T
    
    pesos_unidad_norm_copy = pesos_unidad_norm_copy.sort_values(by=[0], ascending = False)
    pesos_decena_norm_copy = pesos_decena_norm_copy.sort_values(by=[0], ascending = False)
    pesos_centena_norm_copy = pesos_centena_norm_copy.sort_values(by=[0], ascending = False)
    
    pesos_unidad_norm_copy = pesos_unidad_norm_copy.T
    pesos_decena_norm_copy = pesos_decena_norm_copy.T
    pesos_centena_norm_copy = pesos_centena_norm_copy.T
    '''
    pesos_unidad_copy['Total'] = pesos_unidad_copy.sum(axis = 1)
    pesos_decena_copy['Total'] = pesos_decena_copy.sum(axis = 1)
    pesos_centena_copy['Total'] = pesos_centena_copy.sum(axis = 1)
    '''
    
    return [pesos_unidad_copy, pesos_decena_copy, pesos_centena_copy, pesos_unidad_norm_copy, pesos_decena_norm_copy, pesos_centena_norm_copy]

In [72]:
pesos_usuario_test = pesos_completos_usuario('c361c772cd0220bb16dcfb2ad803e4903334ab94')
pd.set_option('max_columns',40)
display(pesos_usuario_test[3])
display(pesos_usuario_test[4])
display(pesos_usuario_test[5])

,720,378,428,726,711,770,155,746,582,584,771,589,725,586,728,458,637,544,535,587,...,362,814,677,537,717,757,599,871,348,950,266,636,593,830,695,590,159,438,341,324
0,0.380952,0.095238,0.095238,0.095238,0.047619,0.047619,0.047619,0.047619,0.02381,0.02381,0.02381,0.02381,0.02381,0.02381,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


,720,370,420,580,770,710,740,150,440,780,600,170,760,660,250,20,640,950,830,670,...,410,540,320,900,880,500,850,930,590,520,730,110,790,450,610,120,680,910,0,30
0,0.5,0.095238,0.095238,0.095238,0.071429,0.047619,0.047619,0.047619,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


,700,500,300,400,100,800,600,900,-900,0,200
0,0.666667,0.095238,0.095238,0.095238,0.047619,0.0,0.0,0.0,0.0,0.0,0.0
